In [1]:
import numpy as np
import xobjects as xo
import xtrack as xt
import xpart as xp
import json
import pandas as pd
from cpymad.madx import Madx
#from matplotlib import pyplot as plt
import NAFFlib
from math import modf
#from scipy.constants import physical_constants
from matplotlib import pyplot as plt
from scipy.stats import linregress

In [2]:
#input
i_mo = -350
p0c = 7000e9
normal_emitt_x = 3e-6
normal_emitt_y = 3e-6
#proton_mass = physical_constants['proton mass energy equivalent in MeV'][0]/1000 

#ctx_gpu = xo.ContextCupy()
ctx_cpu = xo.ContextCpu()

if False:
    mad = Madx()
    mad.option(echo=False)
    mad.call('andrea.madx')
    mad.use(sequence="lhcb1")
    line = xt.Line.from_madx_sequence(mad.sequence['lhcb1'],
                                  deferred_expressions=True
                                  )
    with open('line.json', 'w') as fid:
        json.dump(line.to_dict(), fid, cls=xo.JEncoder)

else:
    with open('line.json', 'r') as fid:
        loaded_dct = json.load(fid)
    line = xt.Line.from_dict(loaded_dct)
particle_0 = xp.Particles(_context=ctx_cpu, p0c=p0c)

line.vars['i_mo'] = i_mo
line.vars['cmrs.b1_sq']=0.5*line.vars['cmrs.b1_sq']


tracker_normal = xt.Tracker(_context=ctx_cpu, line=line)
tw_normal = tracker_normal.twiss(particle_0)
betx_at_ip3 = tw_normal['betx'][0]
bety_at_ip3 = tw_normal['bety'][0]
sigma_x = np.sqrt(betx_at_ip3*normal_emitt_x/(particle_0.gamma0*particle_0.beta0))
sigma_y = np.sqrt(bety_at_ip3*normal_emitt_y/(particle_0.gamma0*particle_0.beta0))
#assert np.isclose(sigma_x,219.93e-6)
#assert np.isclose(sigma_x,219.93e-6)

p0_normal = tw_normal['particle_on_co']
print(p0_normal.x)
print(tw_normal['particle_on_co'].x)


N_particles = 10

particles_normal = xp.build_particles(_context=ctx_cpu,
                                       particle_ref=p0_normal,
                                       x=[mysigma*sigma_x[0] for mysigma
                                         in np.linspace(0.2,2.2,N_particles)])





N=1000

n_turns= 1
my_result = {}
for ii in ['x','px','y','py','zeta','delta']:
    my_result[ii] = []


for ii in range(N):
    tracker_normal.track(particles_normal, num_turns=n_turns,turn_by_turn_monitor=False)
    for jj in ['x','px','y','py','zeta','delta']:
        my_result[jj].append(getattr(particles_normal,jj).copy())       
        #my_result[jj].append(getattr(particles_normal,jj).copy())

for jj in ['x','px','y','py','zeta','delta']:
        my_result[jj] = np.array(my_result[jj])
        my_result[jj] = np.array(my_result[jj])

generating ./e9e6e8ef28874d219a5470aa4ab91325.c
the current directory is '/home/afornara/studies/git_postprocessing'
running build_ext
building 'e9e6e8ef28874d219a5470aa4ab91325' extension
gcc -pthread -B /home/afornara/py/2022_03_04/miniconda/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/afornara/py/2022_03_04/miniconda/include -I/home/afornara/py/2022_03_04/miniconda/include -fPIC -O2 -isystem /home/afornara/py/2022_03_04/miniconda/include -fPIC -I/home/afornara/py/2022_03_04/miniconda/include/python3.9 -c e9e6e8ef28874d219a5470aa4ab91325.c -o ./e9e6e8ef28874d219a5470aa4ab91325.o -std=c99 -O3 -Wno-unused-function
gcc -pthread -B /home/afornara/py/2022_03_04/miniconda/compiler_compat -shared -Wl,-rpath,/home/afornara/py/2022_03_04/miniconda/lib -Wl,-rpath-link,/home/afornara/py/2022_03_04/miniconda/lib -L/home/afornara/py/2022_03_04/miniconda/lib -L/home/afornara/py/2022_03_04/miniconda/lib -Wl,-rpath,/home/afornara/py/2022_03_04/minicon

In [12]:
getattr(particles_normal,'x')

array([ 6.76039821e-05,  9.65368030e-05,  9.78874663e-06, -2.47984254e-04,
       -5.80345100e-04, -5.97891306e-04,  6.19357507e-05,  9.08659308e-04,
        5.07139166e-04, -1.10617951e-03])

In [6]:
len(my_result['x'][:,0])

2000